In [1]:
pip install pulp

  Using cached pulp-3.3.0-py3-none-any.whl.metadata (8.4 kB)
Using cached pulp-3.3.0-py3-none-any.whl (16.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from pathlib import Path
from typing import List, Tuple
import pulp


def parse_line(line: str) -> Tuple[List[List[int]], List[int]]:
    """
    Parse a line like:
    [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
    Returns (buttons, targets)
    - buttons: list of lists of indices (each button's affected counters)
    - targets: list of integers (desired counter values)
    """
    s = line.strip()
    if not s:
        return [], []

    # Find braces part for targets
    lb = s.find("{")
    rb = s.find("}")
    if lb == -1 or rb == -1:
        raise ValueError("Line missing { } targets: " + line)
    targets_str = s[lb + 1 : rb]
    targets = [int(x.strip()) for x in targets_str.split(",") if x.strip() != ""]

    # buttons are the parenthesis groups before the '{'
    before = s[:lb]
    buttons: List[List[int]] = []
    pos = 0
    while True:
        i = before.find("(", pos)
        if i == -1:
            break
        j = before.find(")", i)
        if j == -1:
            break
        inside = before[i + 1 : j].strip()
        if inside == "":
            buttons.append([])
        else:
            idxs = [int(x.strip()) for x in inside.split(",") if x.strip() != ""]
            buttons.append(idxs)
        pos = j + 1

    # We only need the portion of each button that references valid counters (0..len(targets)-1)
    n = len(targets)
    buttons = [[x for x in btn if 0 <= x < n] for btn in buttons]

    return buttons, targets


def solve_machine_ilp(buttons: List[List[int]], targets: List[int]) -> int:
    """
    Formulate and solve:
      minimize sum_j x_j
      subject to: for each counter i: sum_j A[i][j] * x_j == targets[i]
                  x_j >= 0 and integer
    Returns minimal sum(x_j) as int.
    """
    m = len(buttons)
    n = len(targets)

    # Quick infeasibility check: any target>0 with no touching button -> impossible
    for i in range(n):
        if targets[i] > 0:
            touched = False
            for btn in buttons:
                if i in btn:
                    touched = True
                    break
            if not touched:
                raise RuntimeError(
                    f"Counter {i} has target {targets[i]} but no button touches it."
                )

    # Create ILP
    prob = pulp.LpProblem("machine", pulp.LpMinimize)

    # Variables: x_0 .. x_{m-1}, integer >= 0
    x_vars = [pulp.LpVariable(f"x_{j}", lowBound=0, cat="Integer") for j in range(m)]

    # Objective: minimize total presses
    prob += pulp.lpSum(x_vars)

    # Constraints: for each counter i
    for i in range(n):
        # sum over buttons that affect counter i of x_j == targets[i]
        coeffs = []
        vars_for_row = []
        for j in range(m):
            if i in buttons[j]:
                coeffs.append(1)
                vars_for_row.append(x_vars[j])
        # If there are no vars, we already checked infeasibility above
        prob += (pulp.lpSum(vars_for_row) == targets[i], f"counter_{i}")

    # Solve with CBC quietly
    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    status = pulp.LpStatus[prob.status]
    if status != "Optimal":
        raise RuntimeError(f"ILP did not find optimal solution (status={status}).")

    # Retrieve solution and compute sum
    total = 0
    for j in range(m):
        val = x_vars[j].value()
        if val is None:
            raise RuntimeError("Variable has no value in solution.")
        # val may be float-like; cast to int
        iv = int(round(val))
        if iv < 0:
            raise RuntimeError("Negative press in solution.")
        total += iv

    return total


def solve_file(path: str) -> int:
    total_sum = 0
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"{path} not found.")
    with p.open() as fh:
        lineno = 0
        for line in fh:
            lineno += 1
            line = line.strip()
            if not line:
                continue
            buttons, targets = parse_line(line)
            try:
                machine_min = solve_machine_ilp(buttons, targets)
            except Exception as e:
                # Provide helpful error context
                raise RuntimeError(f"Error solving line {lineno}: {e}") from e
            print(f"Line {lineno}: min presses = {machine_min}")
            total_sum += machine_min
    return total_sum


if __name__ == "__main__":
    total = solve_file("../input.txt")
    with open("../output_part_two.txt", 'w') as out_file:
        out_file.write(f"{total}")
    print("Total min presses (Part Two) =", total)


Line 1: min presses = 138
Line 2: min presses = 38
Line 3: min presses = 52
Line 4: min presses = 64
Line 5: min presses = 62
Line 6: min presses = 107
Line 7: min presses = 199
Line 8: min presses = 129
Line 9: min presses = 78
Line 10: min presses = 39
Line 11: min presses = 237
Line 12: min presses = 81
Line 13: min presses = 107
Line 14: min presses = 83
Line 15: min presses = 111
Line 16: min presses = 219
Line 17: min presses = 81
Line 18: min presses = 74
Line 19: min presses = 93
Line 20: min presses = 60
Line 21: min presses = 93
Line 22: min presses = 91
Line 23: min presses = 180
Line 24: min presses = 201
Line 25: min presses = 222
Line 26: min presses = 95
Line 27: min presses = 66
Line 28: min presses = 213
Line 29: min presses = 8
Line 30: min presses = 191
Line 31: min presses = 39
Line 32: min presses = 215
Line 33: min presses = 26
Line 34: min presses = 82
Line 35: min presses = 68
Line 36: min presses = 60
Line 37: min presses = 53
Line 38: min presses = 239
Line 39